Data split

#ตอนแบ่ง train test เช็ค class distribution


In [10]:
import shutil
import os
import numpy as np
import argparse

def get_files_from_folder(path):

    files = os.listdir(path)
    return np.asarray(files)

def main(path_to_data, path_to_test_data, train_ratio):
    # get dirs
    _, dirs, _ = next(os.walk(path_to_data))

    # calculates how many train data per class
    data_counter_per_class = np.zeros((len(dirs)))
    for i in range(len(dirs)):
        path = os.path.join(path_to_data, dirs[i])
        files = get_files_from_folder(path)
        data_counter_per_class[i] = len(files)
    print("data per class", data_counter_per_class)
    test_counter = np.round(data_counter_per_class * (1 - train_ratio))
    print("data per test", test_counter)
    
    # transfers files
    for i in range(len(dirs)):
        path_to_original = os.path.join(path_to_data, dirs[i])
        path_to_save = os.path.join(path_to_test_data, dirs[i])

        #creates dir
        if not os.path.exists(path_to_save):
            os.makedirs(path_to_save)
        files = get_files_from_folder(path_to_original)
                                     
        # random file sequence
        np.random.shuffle(files)
                                      
        # moves data
        for j in range(int(test_counter[i])):
            dst = os.path.join(path_to_save, files[j])
            src = os.path.join(path_to_original, files[j])
            shutil.move(src, dst)

# Path to data
path_to_data = "./data/video/train_ravdess"
# Path to test data where to save
path_to_test_data = "./data/video/test_ravdess"
# Train ratio - 0.7 means splitting data in 70 % train and 30 % test
train_ratio = float(0.7)
main(path_to_data, path_to_test_data, train_ratio)

data per class [32. 32.  0. 32. 32. 16. 16.  0.]
data per test [10. 10.  0. 10. 10.  5.  5.  0.]
